**Multi-paradigm programming in Python**

**Quick Intro**
* Elias Mistler
* Previse
    * Invoice financing
    * based on ML
    * corporate data
    * improve SME cashflow
* Machine Learning Engineer
    * ML integration into invoice processing platform
    * Buyer data intake and mapping
    * Operational tooling

In [8]:
import logging
import random
from itertools import chain, product, starmap
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, List, Callable
from xml.etree import ElementTree

import numpy as np
import requests
import xmltodict
from toolz import *
import math

from collections import Counter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Contents

* Introduction
* Code Structure
* Data Structures
* State Handling
* Multiple implementations
* Summary

# Introduction
* Python = multi-paradigm, pragmatic (unlike OO Java / FP Clojure)
* libraries
* OOP and FP are **concepts**, not tied to syntax (`class` or `def`)

## Object-oriented principles
* mutable data structures
* (relies on rich type system)
* class hierarchies
    * inheritance
    * abstraction
    * encapsulation
    * polymorphism

## Functional programming Principles

* immutable data structures
* (relies on simple data types)
* pure functions
    * no side-effects
    * idempotent

## Sudoku
<img src="./img/sudoku.png" style="width: 400px; float: left"/>

* 9 x 9 field (81 squares)
* numbers from 1 - 9
* each row/column/block should contain each digit

# Code structure: high- vs. low-context
Example: parse raw Sudoku string (from [OpenSudoku](https://opensudoku.moire.org/)) to array

In [9]:
raw_example = '700150000003002097800470126500390200030010050008027001975031004120700900000065002'

## Factory function (OO)

In [10]:
@dataclass
class Sudoku:
    values: np.array

    @classmethod
    def from_string(cls, raw):
        values = []
        for idx, digit in enumerate(raw):
            values.append(int(digit))
        values = np.array(values, dtype='int64').reshape((9, 9))
        return cls(values)

In [11]:
Sudoku.from_string(raw_example)

Sudoku(values=array([[7, 0, 0, 1, 5, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 2, 0, 9, 7],
       [8, 0, 0, 4, 7, 0, 1, 2, 6],
       [5, 0, 0, 3, 9, 0, 2, 0, 0],
       [0, 3, 0, 0, 1, 0, 0, 5, 0],
       [0, 0, 8, 0, 2, 7, 0, 0, 1],
       [9, 7, 5, 0, 3, 1, 0, 0, 4],
       [1, 2, 0, 7, 0, 0, 9, 0, 0],
       [0, 0, 0, 0, 6, 5, 0, 0, 2]]))

* explicit, high-context
* easy to find and use

## Isolated function (FP)

In [12]:
def parse_raw(raw):
    return np.array(list(map(int, raw)), dtype='int64').reshape((9, 9))

In [13]:
parse_raw(raw_example)

array([[7, 0, 0, 1, 5, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 2, 0, 9, 7],
       [8, 0, 0, 4, 7, 0, 1, 2, 6],
       [5, 0, 0, 3, 9, 0, 2, 0, 0],
       [0, 3, 0, 0, 1, 0, 0, 5, 0],
       [0, 0, 8, 0, 2, 7, 0, 0, 1],
       [9, 7, 5, 0, 3, 1, 0, 0, 4],
       [1, 2, 0, 7, 0, 0, 9, 0, 0],
       [0, 0, 0, 0, 6, 5, 0, 0, 2]])

* free of assumptions about the use case
* easy to reuse or generalise

## Multi-paradigm solution
Generalised, low-context pure function, use in high-context class

In [14]:
def parse_raw(raw):
    size = int(math.sqrt(len(raw)))
    return np.array(list(map(int, raw)), dtype='int64').reshape((size, size))
    
    
class Sudoku:
    @classmethod
    def from_string(cls, raw):
        values = parse_raw(raw)
        return cls(values)

* low-context pure functions *and* high-context class
* tidy, reusable code
* generalises well
* works in any context
* easy to use

## That tedious `for`-loop

In [16]:
values = []
for digit in raw_example:
    values.append(int(digit))

values[:5]

[7, 0, 0, 1, 5]

<img src="https://www.profkrg.com/wp-content/uploads/2014/10/I-would-have-written-a-shorter-letter.png" style="max-height: 600px; float: left"/>

* comparatively far from high-level intention
* error prone 
* easy to write
* tedious to read and reconstruct

The alternative:

In [15]:
values = tuple(map(int, raw_example))

values[:5]

(7, 0, 0, 1, 5)

In [17]:
values = thread_last(raw_example, (map, int), tuple)

* concise
* reflects the intention
* easy to read
* can take longer to write

## Further example - display/format

### Object-oriented
Implement `__repr__`

In [18]:
from sudoku.oo.base import Sudoku

Sudoku.from_string(raw_example)

+---+---+---+---+---+---+---+---+---+
| 7 |   |   | 1 | 5 |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   | 3 |   |   | 2 |   | 9 | 7 |
+---+---+---+---+---+---+---+---+---+
| 8 |   |   | 4 | 7 |   | 1 | 2 | 6 |
+---+---+---+---+---+---+---+---+---+
| 5 |   |   | 3 | 9 |   | 2 |   |   |
+---+---+---+---+---+---+---+---+---+
|   | 3 |   |   | 1 |   |   | 5 |   |
+---+---+---+---+---+---+---+---+---+
|   |   | 8 |   | 2 | 7 |   |   | 1 |
+---+---+---+---+---+---+---+---+---+
| 9 | 7 | 5 |   | 3 | 1 |   |   | 4 |
+---+---+---+---+---+---+---+---+---+
| 1 | 2 |   | 7 |   |   | 9 |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   | 6 | 5 |   |   | 2 |
+---+---+---+---+---+---+---+---+---+

### Functional
explicit functions

In [19]:
from sudoku.fp.load import *

thread_last(raw_example, parse_raw, format_sudoku, print)

+---+---+---+---+---+---+---+---+---+
| 7 |   |   | 1 | 5 |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   | 3 |   |   | 2 |   | 9 | 7 |
+---+---+---+---+---+---+---+---+---+
| 8 |   |   | 4 | 7 |   | 1 | 2 | 6 |
+---+---+---+---+---+---+---+---+---+
| 5 |   |   | 3 | 9 |   | 2 |   |   |
+---+---+---+---+---+---+---+---+---+
|   | 3 |   |   | 1 |   |   | 5 |   |
+---+---+---+---+---+---+---+---+---+
|   |   | 8 |   | 2 | 7 |   |   | 1 |
+---+---+---+---+---+---+---+---+---+
| 9 | 7 | 5 |   | 3 | 1 |   |   | 4 |
+---+---+---+---+---+---+---+---+---+
| 1 | 2 |   | 7 |   |   | 9 |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   | 6 | 5 |   |   | 2 |
+---+---+---+---+---+---+---+---+---+


### Multi-paradigm
<img src="./img/why-not-both.jpg" style="width: 200px;"/>

In [20]:
def format_sudoku(grid):
    ...


class Sudoku:
    ...
    
    def __repr__(self):
        return format_sudoku(self.grid)

# Data Structures: explicit vs. minimalist
Example: The Sudoku grid

## Class Hierarchy (OO)

<img src="./img/erm.png" style="max-height: 250px;"/>

In [21]:
from sudoku.oo.base import *

oo_game = Sudoku.from_string(raw_example)
oo_game.get_row(8)

|   |   |   |   | 6 | 5 |   |   | 2 |

In [22]:
oo_game.get_square(8, 4)

Square(y=8, x=4, digit=6, locked=True)

* assumes certain usage patterns
* intuitive to explore
* fairly rigid
* requires lots of boilerplate

In [23]:
# even with `dataclass` and without many getters, setters etc:
!wc ./sudoku/oo/base.py

     121     335    3148 ./sudoku/oo/base.py


## Simplicity (FP)

In [25]:
import schema

sudoku_schema = schema.And(np.ndarray,
                           lambda a: a.shape == (9, 9),
                           lambda a: a.dtype == 'int64')

In [26]:
thread_last(raw_example, parse_raw, sudoku_schema.validate)

array([[7, 0, 0, 1, 5, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 2, 0, 9, 7],
       [8, 0, 0, 4, 7, 0, 1, 2, 6],
       [5, 0, 0, 3, 9, 0, 2, 0, 0],
       [0, 3, 0, 0, 1, 0, 0, 5, 0],
       [0, 0, 8, 0, 2, 7, 0, 0, 1],
       [9, 7, 5, 0, 3, 1, 0, 0, 4],
       [1, 2, 0, 7, 0, 0, 9, 0, 0],
       [0, 0, 0, 0, 6, 5, 0, 0, 2]])

* minimalist approach with basic data types
* zero boilerplate
* no context on the data structure itself (harder to explore)

## Multi-paradigm solution

In [27]:
@dataclass
class Sudoku:
    grid: np.ndarray
    
    @property
    def remaining_blanks(self):
        return (self.grid == 0).sum()
    
    def __repr__(self):
        ...

* "shallow" class
* saves a lot of boilerplate code
* adds context for user

# State handling - mutable vs. immutable
Example: Fill digits into Sudoku

Using a multi-paradigm implementation, inspired by `pandas`:

In [28]:
from sudoku.mp.base import Sudoku

blank = 81 * '0'
sudoku = Sudoku.from_string(blank)
sudoku

+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+

## Mutable (OO)

In [29]:
sudoku.set_digit(0, 0, 7, inplace=True)
sudoku

+---+---+---+---+---+---+---+---+---+
| 7 |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+

* tiny performance benefit (usually negligible)

## Immutable (FP)

In [34]:
sudoku.set_digit(2, 2, 4, inplace=False)

+---+---+---+---+---+---+---+---+---+
| 7 |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   | 4 |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+

In [35]:
sudoku

+---+---+---+---+---+---+---+---+---+
| 7 |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+

* easy to reuse or paralellise
* natural versioning
* lends itself well to pipelines or method chaining

### Method Chaining

In [38]:
(sudoku
 .set_digit(2, 8, 9)
 .set_digit(1, 0, 9)
 .set_digit(0, 3, 9))

+---+---+---+---+---+---+---+---+---+
| 7 |   |   | 9 |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
| 9 |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   | 9 |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+---+

## Recommendation
* make use of immutable data structures like `@dataclass(frozen=True)`, `frozendict`, `NamedTuple` and `pyrsistent.pmap`)
* use mutable data structures in immutable ways (try the `toolz` library!)
* keep functions pure and idempotent - use classes where configuration and state is required

*Note*: "functions" with side effects and global variables = **procedures** (avoid...)

## Example in Pandas

In [39]:
import pandas as pd

df = pd.DataFrame(np.random.random((5,3)), columns=list('abc'))
df

,a,b,c
0,0.242487,0.987940,0.801766
1,0.365565,0.761565,0.562954
2,0.024770,0.709967,0.957397
3,0.303059,0.140491,0.838389
4,0.846308,0.689011,0.222807


In [43]:
(df
 .assign(sum=lambda df: df.sum(axis=1))
 .assign(a_percent=lambda df: df['a'] / df['sum'])
 .drop(index=[1,3]))

,a,b,c,sum,a_percent
0,0.242487,0.987940,0.801766,2.032193,0.119323
2,0.024770,0.709967,0.957397,1.692134,0.014638
4,0.846308,0.689011,0.222807,1.758127,0.481369


In [41]:
df

,a,b,c
0,0.242487,0.987940,0.801766
1,0.365565,0.761565,0.562954
2,0.024770,0.709967,0.957397
3,0.303059,0.140491,0.838389
4,0.846308,0.689011,0.222807


* cleaner Jupyter notebooks (execution order...)
* better reusability
* close to production-ready

# Multiple implementations: polymorphism vs. function composition
Example: Different Sudoku solvers

* Deterministic (mask, fill unambiguous, repeat) - insufficient

* Random (mask, fill random, repeat) - prohibitively slow

* Combined (deterministic as much as possible, random step, repeat)

## OO - Solver class hierarchy

<img src="./img/erm_solver.png" style="max-height: 800px"/>

In [49]:
from sudoku.oo.solver import *

sudoku = Sudoku.from_string(raw_example)
solver = DeterministicSolver(sudoku)
solver.solve()

sudoku

+---+---+---+---+---+---+---+---+---+
| 7 | 6 | 2 | 1 | 5 | 9 | 4 | 8 | 3 |
+---+---+---+---+---+---+---+---+---+
| 4 | 1 | 3 | 6 | 8 | 2 | 5 | 9 | 7 |
+---+---+---+---+---+---+---+---+---+
| 8 | 5 | 9 | 4 | 7 | 3 | 1 | 2 | 6 |
+---+---+---+---+---+---+---+---+---+
| 5 | 4 | 1 | 3 | 9 | 6 | 2 | 7 | 8 |
+---+---+---+---+---+---+---+---+---+
| 2 | 3 | 7 | 8 | 1 | 4 | 6 | 5 | 9 |
+---+---+---+---+---+---+---+---+---+
| 6 | 9 | 8 | 5 | 2 | 7 | 3 | 4 | 1 |
+---+---+---+---+---+---+---+---+---+
| 9 | 7 | 5 | 2 | 3 | 1 | 8 | 6 | 4 |
+---+---+---+---+---+---+---+---+---+
| 1 | 2 | 6 | 7 | 4 | 8 | 9 | 3 | 5 |
+---+---+---+---+---+---+---+---+---+
| 3 | 8 | 4 | 9 | 6 | 5 | 7 | 1 | 2 |
+---+---+---+---+---+---+---+---+---+

* Single-method classes seem excessive and cause boilerplate
* Complicated design for simple functionality

In [50]:
!wc ./sudoku/oo/solver.py

     130     344    4121 ./sudoku/oo/solver.py


## FP - solving function composition

<img src="./img/fp_solve.png" style="max-height: 600px;"/>

In [55]:
from sudoku.fp.solve import *
from sudoku.fp.load import *

solve_combined = partial(solve, step_function=combined_step)

thread_last(raw_example, parse_raw, solve_combined, format_sudoku, print)

+---+---+---+---+---+---+---+---+---+
| 7 | 6 | 2 | 1 | 5 | 9 | 4 | 8 | 3 |
+---+---+---+---+---+---+---+---+---+
| 4 | 1 | 3 | 6 | 8 | 2 | 5 | 9 | 7 |
+---+---+---+---+---+---+---+---+---+
| 8 | 5 | 9 | 4 | 7 | 3 | 1 | 2 | 6 |
+---+---+---+---+---+---+---+---+---+
| 5 | 4 | 1 | 3 | 9 | 6 | 2 | 7 | 8 |
+---+---+---+---+---+---+---+---+---+
| 2 | 3 | 7 | 8 | 1 | 4 | 6 | 5 | 9 |
+---+---+---+---+---+---+---+---+---+
| 6 | 9 | 8 | 5 | 2 | 7 | 3 | 4 | 1 |
+---+---+---+---+---+---+---+---+---+
| 9 | 7 | 5 | 2 | 3 | 1 | 8 | 6 | 4 |
+---+---+---+---+---+---+---+---+---+
| 1 | 2 | 6 | 7 | 4 | 8 | 9 | 3 | 5 |
+---+---+---+---+---+---+---+---+---+
| 3 | 8 | 4 | 9 | 6 | 5 | 7 | 1 | 2 |
+---+---+---+---+---+---+---+---+---+


* very clear responsibilities per function
* simple, pragmatic design
* easy to introspect
* much more concise (*and* no base module!)

In [57]:
!wc ./sudoku/fp/solve.py

      98     289    2711 ./sudoku/fp/solve.py


## Multi-paradigm solution

In [77]:
from sudoku.fp import solve as _fp_solve
from sudoku.mp.base import Sudoku


def solve(sudoku: Sudoku, step_function: Callable, max_tries: int = 1):
    if max_tries == 1:
        solved_grid = _fp_solve.solve(sudoku.grid, step_function)
    else:
        solved_grid = _fp_solve.repeat_solve(sudoku.grid,
                                             partial(_fp_solve.solve, step_function=step_function),
                                             max_tries=max_tries)
    return Sudoku(solved_grid)

* simplicity and clarity of FP
* takes and returns high-context Sudoku objects

Or, with more context:

In [78]:
@dataclass(frozen=True)
class Solver:
    step_function: Callable
    max_tries: int = 1

    def __call__(self, sudoku: Sudoku):
        return solve(sudoku, self.step_function, self.max_tries)

*Note*: This can equally be achieved with `functools.partial` or `toolz.curry`

In [79]:
thread_last(raw_example, 
            Sudoku.from_string, 
            Solver(combined_step, max_tries=100))

+---+---+---+---+---+---+---+---+---+
| 7 | 6 | 2 | 1 | 5 | 9 | 4 | 8 | 3 |
+---+---+---+---+---+---+---+---+---+
| 4 | 1 | 3 | 6 | 8 | 2 | 5 | 9 | 7 |
+---+---+---+---+---+---+---+---+---+
| 8 | 5 | 9 | 4 | 7 | 3 | 1 | 2 | 6 |
+---+---+---+---+---+---+---+---+---+
| 5 | 4 | 1 | 3 | 9 | 6 | 2 | 7 | 8 |
+---+---+---+---+---+---+---+---+---+
| 2 | 3 | 7 | 8 | 1 | 4 | 6 | 5 | 9 |
+---+---+---+---+---+---+---+---+---+
| 6 | 9 | 8 | 5 | 2 | 7 | 3 | 4 | 1 |
+---+---+---+---+---+---+---+---+---+
| 9 | 7 | 5 | 2 | 3 | 1 | 8 | 6 | 4 |
+---+---+---+---+---+---+---+---+---+
| 1 | 2 | 6 | 7 | 4 | 8 | 9 | 3 | 5 |
+---+---+---+---+---+---+---+---+---+
| 3 | 8 | 4 | 9 | 6 | 5 | 7 | 1 | 2 |
+---+---+---+---+---+---+---+---+---+

# Key Takeaways

## Object-orientation
* "top-down" design
* larger, topical structures
* functionality and data intertwined
* explicit, high-context

leads to:

* intuitive use cases
* high explorability

## Functional programming
* "bottom-up" design
* simplistic thinking
* small chunks of reusable logic, separate from data
* high isolation, low context

leads to
* high reusability
* tidy, concise code
* flexible use cases

## Multi-paradigm programming
*pick & mix* of both worlds:
* pure functions in mutable context
    * brings the simplicity and elegance of FP into OO
    * make your code explorable and easy to understand
    * *remember*: no side effects, no problem!
* mutable data in immutable context
    * use your favourite OO libaries in concise FP code
    * *remember*: copy-and-modify mutable data structures!

leads to (ideally) - best of both worlds:
* intuitive **and** flexible use cases
* high explorability **and** reusability

## My preferred Approach
* iterate with a REPL
* use immutable data types and pure functions where possible
* create classes where either:
    * required due to syntax or library
    * high-context use cases are required

# Thank you for your attention!

# References & Further Reading
* https://www.researchgate.net/publication/2740355_Multiparadigm_Patterns_of_Thought_and_Design 
* https://www.oodesign.com/
* https://patternsinfp.wordpress.com/
* https://www.freecodecamp.org/news/an-introduction-to-the-basic-principles-of-functional-programming-a2c2a15c84/
* https://introprogramming.info/english-intro-csharp-book/read-online/chapter-20-object-oriented-programming-principles/#:~:text=In%20this%20chapter%20we%20will,implementation%2C%20polymorphism%20and%20virtual%20methods.
* https://toolz.readthedocs.io/en/latest/
* https://docs.python.org/3/library/dataclasses.html
* https://pypi.org/project/schema/
* https://www.codenewbie.org/blogs/object-oriented-programming-vs-functional-programming#:~:text=OOP%20says%20that%20bringing%20together,be%20kept%20separate%20for%20clarity.
* https://wiki.c2.com/?ClassicOoAntiPatterns
* https://www.drdobbs.com/cpp/how-non-member-functions-improve-encapsu/184401197
* https://opensudoku.moire.org/
* https://treyhunner.com/2019/04/is-it-a-class-or-a-function-its-a-callable/
* https://blog.miguelgrinberg.com/post/the-ultimate-guide-to-python-decorators-part-i-function-registration
* https://realpython.com/primer-on-python-decorators/
* https://medium.com/@shivama205/dependency-injection-python-cb2b5f336dce